In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Locate the data
data = '../Nikkei Std.csv'
# Read data with pandas
dataset = pd.read_csv(data)
# Extract the values of the desired column
full_set = dataset['NIKKEI_225'].values
dataset

,Unnamed: 0,Date,NIKKEI_225,NIK_Returns_Rolling_Std
0,0,1979-12-28,0.622472,NaN
1,1,1980-01-04,-0.141717,NaN
2,2,1980-01-07,-0.114483,NaN
3,3,1980-01-08,0.162223,NaN
4,4,1980-01-09,0.435000,NaN
...,...,...,...,...
10113,10113,2021-02-08,2.117191,0.959234
10114,10114,2021-02-09,0.399577,0.958767
10115,10115,2021-02-10,0.193182,0.953750
10116,10116,2021-02-12,-0.144977,0.954752


In [3]:
# We split our data on the 7200 mark as it represents 70% of it
train_data = full_set[5061:9061]
test_data = full_set[9061:]
# dataset.iloc[5061]

In [4]:
# Data Normalization
from sklearn.preprocessing import MinMaxScaler
# Data must be reshaped in order to use fit_transform
training_set = train_data.reshape(-1,1)
test_set = test_data.reshape(-1,1)

sc = MinMaxScaler(feature_range=(0,1))

X_train_scaled = sc.fit_transform(training_set)

In [5]:
# Create lists for x and y train
X_train = []

y_train = []

# We will run our model on 100 timesteps
for i in range(20,4000):
    
    X_train.append(X_train_scaled[i- 20:i,0])

    y_train.append(X_train_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [6]:
# Import the necessary models from keras within tensor flow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
import tensorflow as tf

In [7]:
# Create a callback function with keras, which will monitor thr rmse of each epoch 
# And stop our model on the epoch that offers the best balance so we don't overfit our model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [8]:
# Create the layers of our model droping out and replenishing 20% of our data after each layer
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
# Include a printout of the model summary 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 20, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

In [9]:
model.compile(optimizer='adam',loss= 'mean_squared_error',
             metrics=[tf.keras.metrics.RootMeanSquaredError()])
# While we git our model we assign it to a variable 
history = model.fit(X_train,y_train,epochs=20,batch_size=100,callbacks=[callback], verbose = 2)

Epoch 1/20
40/40 - 16s - loss: 0.0240 - root_mean_squared_error: 0.1549
Epoch 2/20
40/40 - 3s - loss: 0.0062 - root_mean_squared_error: 0.0787
Epoch 3/20
40/40 - 3s - loss: 0.0057 - root_mean_squared_error: 0.0755
Epoch 4/20
40/40 - 3s - loss: 0.0053 - root_mean_squared_error: 0.0727
Epoch 5/20
40/40 - 3s - loss: 0.0052 - root_mean_squared_error: 0.0721
Epoch 6/20
40/40 - 3s - loss: 0.0050 - root_mean_squared_error: 0.0704
Epoch 7/20
40/40 - 3s - loss: 0.0050 - root_mean_squared_error: 0.0708
Epoch 8/20
40/40 - 3s - loss: 0.0049 - root_mean_squared_error: 0.0701
Epoch 9/20
40/40 - 3s - loss: 0.0049 - root_mean_squared_error: 0.0703
Epoch 10/20
40/40 - 3s - loss: 0.0049 - root_mean_squared_error: 0.0700
Epoch 11/20
40/40 - 3s - loss: 0.0048 - root_mean_squared_error: 0.0696
Epoch 12/20
40/40 - 3s - loss: 0.0048 - root_mean_squared_error: 0.0696
Epoch 13/20
40/40 - 3s - loss: 0.0048 - root_mean_squared_error: 0.0692
Epoch 14/20
40/40 - 3s - loss: 0.0047 - root_mean_squared_error: 0.0689


In [10]:
# Print out the number of epochs used for this model
print(len(history.history['loss']))

20


In [12]:
# Making Predictions to the test set
dataset_total = np.concatenate((train_data, test_data), axis = 0)
# Prepare our data to the test
inputs = dataset_total[len(dataset_total) - len(test_data) - 100:]
# Reshape our data to fit the upcoming for loop
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

# Create lists to which we'll append x and y values for future test
X_test = []
y_test = []

# Create a loop that will run through groups of 200 timesteps for the test sample
for i in range(20, 1157):
    X_test.append(inputs[i-20:i,0])
    y_test.append(inputs[i, 0])

In [13]:
# Turn our lists into a numpy numbers array
X_test = np.array(X_test)
y_test = np.array(y_test)

# Reshape our array into 3 dimensions so it can fit the LSTM Model
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

y_test = y_test.reshape(-1, 1)

In [14]:
 model_loss, rmse = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Root Mean Square Error:{rmse}")

36/36 - 5s - loss: 0.0022 - root_mean_squared_error: 0.0469
Normal Neural Network - Loss: 0.0022015508729964495, Root Mean Square Error:0.04692068696022034


In [15]:
predicted_return = model.predict(X_test)
predicted_return = sc.inverse_transform(predicted_return)

In [17]:
predicted_return

array([[-0.02039594],
       [-0.02416229],
       [-0.03688791],
       ...,
       [ 0.01966981],
       [ 0.0231932 ],
       [ 0.02395409]], dtype=float32)

In [18]:
y_test = sc.inverse_transform(y_test)


In [23]:
predicted_return = predicted_return * 100


In [24]:
import plotly.express as px

In [25]:
output = pd.DataFrame({
    'predicted_return' : [x[0] for x in predicted_return],
    'actual_return' : [x[0] for x in y_test]
})
output

,predicted_return,actual_return
0,-2.039594,1.276585
1,-2.416229,-0.639433
2,-3.688791,1.074523
3,-3.915686,-7.921557
4,-4.159033,2.388911
...,...,...
1132,3.916497,2.117191
1133,2.948365,0.399577
1134,1.966981,0.193182
1135,2.319320,-0.144977


In [26]:
px.line(output, y=["predicted_return", "actual_return"])

In [29]:
output.to_csv('../../Model Results/Nikkei-Returns-Predicted.csv')
